<a href="https://colab.research.google.com/github/Lucas-Pas/BootCampDIO/blob/main/Recomenda%C3%A7%C3%A3o_por_Imagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow opencv-python scikit-learn numpy matplotlib

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model

In [ ]:
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    """
    Carrega e pré-processa uma imagem para o formato adequado para o modelo.
    """
    img = cv2.imread(img_path)
    img = cv2.resize(img, target_size)  # Redimensiona a imagem
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Converte para RGB
    img = preprocess_input(img)  # Pré-processamento específico para ResNet50
    return img

In [ ]:
def load_feature_extractor():
    """
    Carrega o modelo ResNet50 pré-treinado para extração de características.
    """
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    model = Model(inputs=base_model.input, outputs=base_model.output)
    return model

In [ ]:
def extract_features(model, img_array):
    """
    Extrai características de uma imagem usando o modelo.
    """
    img_array = np.expand_dims(img_array, axis=0)  # Adiciona dimensão do batch
    features = model.predict(img_array)
    return features.flatten()  # Retorna um vetor unidimensional

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
def process_dataset(dataset_path):
    """
    Processa todas as imagens no dataset e extrai suas características.
    """
    model = load_feature_extractor()
    features_dict = {}

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                img_path = os.path.join(root, file)
                img = load_and_preprocess_image(img_path)
                features = extract_features(model, img)
                features_dict[img_path] = features

    return features_dict

# Caminho para o dataset (ajuste conforme necessário)
dataset_path = "dataset"
features_dict = process_dataset(dataset_path)

In [ ]:
np.save("features_dict.npy", features_dict)

In [ ]:
features_dict = np.load("features_dict.npy", allow_pickle=True).item()

In [ ]:
def recommend_similar_images(query_img_path, features_dict, top_n=5):
    """
    Recomenda as imagens mais semelhantes à imagem de consulta.
    """
    model = load_feature_extractor()
    query_img = load_and_preprocess_image(query_img_path)
    query_features = extract_features(model, query_img)

    similarities = {}
    for img_path, features in features_dict.items():
        sim = cosine_similarity([query_features], [features])[0][0]
        similarities[img_path] = sim

    # Ordena as imagens por similaridade
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    return sorted_similarities[:top_n]

In [ ]:
from google.colab import files
uploaded = files.upload()

# Obter o caminho da imagem de consulta
query_img_path = list(uploaded.keys())[0]

In [ ]:
recommendations = recommend_similar_images(query_img_path, features_dict, top_n=5)

print("Imagens recomendadas:")
for img_path, similarity in recommendations:
    print(f"Imagem: {img_path}, Similaridade: {similarity:.4f}")

In [ ]:
def display_images(query_img_path, recommendations):
    """
    Exibe a imagem de consulta e as imagens recomendadas.
    """
    plt.figure(figsize=(15, 10))

    # Exibir a imagem de consulta
    plt.subplot(1, len(recommendations) + 1, 1)
    query_img = cv2.imread(query_img_path)
    query_img = cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB)
    plt.imshow(query_img)
    plt.title("Imagem de Consulta")
    plt.axis("off")

    # Exibir as imagens recomendadas
    for i, (img_path, similarity) in enumerate(recommendations):
        plt.subplot(1, len(recommendations) + 1, i + 2)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.title(f"Similaridade: {similarity:.4f}")
        plt.axis("off")

    plt.show()

# Exibir resultados
display_images(query_img_path, recommendations)